In [1]:
from __future__ import print_function

__author__ = 'maxim'

import numpy as np
import gensim
import string

from keras.callbacks import LambdaCallback
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.utils.data_utils import get_file

Using TensorFlow backend.


In [15]:
print('\nPreparing the sentences...')
max_sentence_len = 40
with open('/content/Extractive-based-summarization/rahul dravid.txt','r') as file_:
  docs = file_.readlines()
sentences = [[word for word in doc.lower().translate(string.punctuation).split()[:max_sentence_len]] for doc in docs]
print('Num sentences:', len(sentences))


Preparing the sentences...
Num sentences: 113


In [19]:
print('\nTraining word2vec...')
word_model = gensim.models.Word2Vec(sentences, size=100, min_count=1, window=5, iter=100)
pretrained_weights = word_model.wv.syn0
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)
print('Checking similar words:')
for word in [ 'playing', 'early']:
  most_similar = ', '.join('%s (%.2f)' % (similar, dist) for similar, dist in word_model.most_similar(word)[:8])
print('  %s -> %s' % (word, most_similar))


Training word2vec...
Result embedding shape: (782, 100)
Checking similar words:
  early -> under-15, (1.00), karnataka (1.00), bowling.[153] (1.00), swing (1.00), represented (1.00), 5 (1.00), 12, (1.00), fast (1.00)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
def word2idx(word):
  return word_model.wv.vocab[word].index
def idx2word(idx):
  return word_model.wv.index2word[idx]


In [21]:
print('\nPreparing the data for LSTM...')
train_x = np.zeros([len(sentences), max_sentence_len], dtype=np.int32)
train_y = np.zeros([len(sentences)], dtype=np.int32)
for i, sentence in enumerate(sentences):
  for t, word in enumerate(sentence[:-1]):
    train_x[i, t] = word2idx(word)
  train_y[i] = word2idx(sentence[-1])
print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)



Preparing the data for LSTM...
train_x shape: (113, 40)
train_y shape: (113,)


In [22]:
print('\nTraining LSTM...')
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights]))
model.add(LSTM(units=emdedding_size))
model.add(Dense(units=vocab_size))
model.add(Activation('softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')


Training LSTM...


In [0]:
def sample(preds, temperature=1.0):
  if temperature <= 0:
    return np.argmax(preds)
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [0]:
def generate_next(text, num_generated=10):
  word_idxs = [word2idx(word) for word in text.lower().split()]
  for i in range(num_generated):
    prediction = model.predict(x=np.array(word_idxs))
    idx = sample(prediction[-1], temperature=0.7)
    word_idxs.append(idx)
  return ' '.join(idx2word(idx) for idx in word_idxs)


In [0]:
def on_epoch_end(epoch, _):
  print('\nGenerating text after epoch: %d' % epoch)
  texts = [
    'playing',
    'karnataka',
    'born',
    'a',
  ]
  for text in texts:
    sample = generate_next(text)
    print('%s... -> %s' % (text, sample))

In [28]:
model.fit(train_x, train_y,
          batch_size=128,
          epochs=20,
callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])

Epoch 1/20
113/113 [==============================] - 0s 1ms/step - loss: 6.5750

Generating text after epoch: 0
playing... -> playing winning 208 century success league australia), from scored national everything
karnataka... -> karnataka odi strategy fee calendar games arrived day, chennai kept fee
born... -> born day, found indian renaissance 1999.[41] 0–2 contribute six, his vaughan.[50][156]+
a... -> a game. involving lost team, college.[32] when bangalore, sharad sharjah since,
Epoch 2/20
113/113 [==============================] - 0s 880us/step - loss: 6.4858

Generating text after epoch: 1
playing... -> playing stage, college.[32] abandoned when fortunes some resignation team, lanka 39-run
karnataka... -> karnataka debut contributing himself test, kumble league 1999 performance, fifties.[58] defeats
born... -> born zimbabwe whitewash aiwa wicket-keeper decide themselves outing whole provided season
a... -> a of game nail 2014, came fifth hundreds sabina seven zealand.
Epoch 3/20